In [1]:
import pandas as pd
import os 
import numpy as np
import torch
import copy
import random

In [2]:
from src.fun import split_data, batch_generator, get_data

In [3]:
config = {'bs': 8,
          'n_layers': 6,
          'd_model': 10,
          'nhead':16,
          'sequence_length': 10,
          'tgt_key': 'rule_closeup',
          'forbidden_keys': ['date',
                             'up']}

data = get_data(config)
training_data, valid_data = split_data(data)
loader = batch_generator(data=training_data)
src, tgt = next(loader)

In [4]:
src, tgt

(tensor([[[ 4.3538e-03,  3.2871e-03,  7.8511e-04,  ...,  1.2378e+00,
            1.2357e+00,  1.2351e+00],
          [ 4.6653e-03,  3.0209e-03,  1.8746e-03,  ...,  1.2395e+00,
            1.2366e+00,  1.2356e+00],
          [ 7.6878e-03,  5.4342e-03,  5.3334e-03,  ...,  1.2453e+00,
            1.2394e+00,  1.2373e+00],
          ...,
          [-6.5417e-03,  2.3915e-03,  5.9509e-03,  ...,  1.2459e+00,
            1.2467e+00,  1.2429e+00],
          [-5.9633e-03, -1.3806e-03,  2.9976e-03,  ...,  1.2430e+00,
            1.2452e+00,  1.2425e+00],
          [-4.0464e-03, -3.6851e-03,  1.3438e-03,  ...,  1.2411e+00,
            1.2440e+00,  1.2422e+00]],
 
         [[-8.1503e-04, -6.1071e-04, -5.8961e-04,  ...,  1.2653e+00,
            1.2659e+00,  1.2659e+00],
          [-6.0604e-03, -3.7044e-03, -2.6118e-03,  ...,  1.2600e+00,
            1.2637e+00,  1.2648e+00],
          [-7.7480e-03, -4.0828e-03, -3.6883e-03,  ...,  1.2564e+00,
            1.2622e+00,  1.2639e+00],
          ...,
    